# CCM Final Project
#### Baysian Modeling of Orientation WM prior

### Content
1. Load and Sort data
2. Prior Model Simulate
3. Fit Prior Model
4. Validation of Prior Model (Recover the params from the simulated data)

### 1. Load and Sort Data

In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize

In [17]:
curr_dif = os.getcwd()
data_dir = os.path.abspath(os.path.join(curr_dif, '..\\..\\'))
os.listdir(data_dir)

['.Rhistory',
 '.Rproj.user',
 'CCM_finalProject',
 'CCM_Final_23Spring.Rproj',
 'data',
 'data_beh.csv',
 'figures',
 'scripts']

In [18]:
data_path = os.path.join(data_dir,'data_beh.csv')
data = pd.read_csv(data_path,sep=',')

data.head()

,subjID,run,trial,phase,oriRef,oriJitt,oriFinal,oriRespInit,oriRespFinal,error,points,rt,outoftime,iti
0,1,1,1,9,2,-9,36,63,62,-26,48,2.609021,NaN,0.2
1,1,1,2,3,1,-9,171,85,165,6,88,3.047162,NaN,0.2
2,1,1,3,4,2,-1,44,58,53,-9,82,2.500657,NaN,0.2
3,1,1,4,1,3,6,96,12,99,-3,94,3.697208,NaN,0.2
4,1,1,5,3,4,3,138,43,143,-5,90,3.665645,NaN,0.2


In [19]:
data_df = pd.DataFrame(data)
flag = data_df['outoftime']!=1
print(sum(flag))
data_clean = data_df[flag]
data_clean['oriRef2'] = (data_clean['oriRef']-1) * 45
data_clean['oriFinal2'] = data_clean['oriRef2'] + data_clean['oriJitt']
data_clean['oriRespFinal2'] = data_clean['oriFinal2'] - data_clean['error']

data_clean

2319


C:\Users\kay\AppData\Local\Temp\ipykernel_24304\3813394554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['oriRef2'] = (data_clean['oriRef']-1) * 45
C:\Users\kay\AppData\Local\Temp\ipykernel_24304\3813394554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['oriFinal2'] = data_clean['oriRef2'] + data_clean['oriJitt']
C:\Users\kay\AppData\Local\Temp\ipykernel_24304\3813394554.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,subjID,run,trial,phase,oriRef,oriJitt,oriFinal,oriRespInit,oriRespFinal,error,points,rt,outoftime,iti,oriRef2,oriFinal2,oriRespFinal2
0,1,1,1,9,2,-9,36,63,62,-26,48,2.609021,NaN,0.2,45,36,62
1,1,1,2,3,1,-9,171,85,165,6,88,3.047162,NaN,0.2,0,-9,-15
2,1,1,3,4,2,-1,44,58,53,-9,82,2.500657,NaN,0.2,45,44,53
3,1,1,4,1,3,6,96,12,99,-3,94,3.697208,NaN,0.2,90,96,99
4,1,1,5,3,4,3,138,43,143,-5,90,3.665645,NaN,0.2,135,138,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2394,15,1,155,7,3,-1,89,93,88,1,98,2.375790,NaN,0.2,90,89,88
2395,15,1,156,1,3,0,90,18,91,-1,98,2.757365,NaN,0.2,90,90,91
2397,15,1,158,5,4,1,136,149,127,9,82,2.387765,NaN,0.2,135,136,127
2398,15,1,159,2,2,-6,39,20,46,-7,86,1.181797,NaN,0.2,45,39,46


### Prior Model Simulate

In [20]:
def prior_simulate_data(targ_loc,prior_mu,prior_std,likelihood_std):
    
    posterior = [];
    answers = [];
    xs = np.linspace(-90, 90, 361)
    for i in range(targ_loc.shape[0]):
        representation = norm.pdf(xs, loc=targ_loc[i], scale=likelihood_std)
        prior = norm.pdf(xs, loc=prior_mu[i], scale=prior_std)
        post = representation * prior
        post = post/sum(post) # normalize
        
        random_answer = np.random.choice(xs,p = post)
        answers.append(random_answer)
        posterior.append(post[round(random_answer)])
        #errors = answers-targ_loc
    return answers,posterior

In [21]:
# Kay
# Also if there is no prior
def no_prior_simulate_data(targ_loc, likelihood_std):
    posterior = []
    answers = []
    xs = np.linspace(-90, 90, 361)
    
    # Uniform prior
    prior = np.ones_like(xs) / len(xs)
    
    for i in range(targ_loc.shape[0]):
        # Likelihood (representation) using a normal distribution
        representation = norm.pdf(xs, loc=targ_loc[i], scale=likelihood_std)
        
        # Posterior is the product of the likelihood and the uniform prior
        post = representation * prior
        post = post / sum(post)  # Normalize
        
        # Sample a random answer from the possible orientations
        random_answer = np.random.choice(xs, p=post)
        answers.append(random_answer)
        posterior.append(post[round(random_answer)])
        
    return answers, posterior

### 3.Fit Prior Model

In [28]:
# Kay
def prior_negative_log_likelihood(targ_loc, real_response, prior_mu, prior_std, likelihood_std):
    nll = 0
    #xs = np.arange(-45, 135, 0.01)
    xs = np.linspace(-90, 90, 361)
    for i in range(targ_loc.shape[0]):
        # Calculate the likelihood
        likelihood = norm.logpdf(real_response[i], loc=targ_loc[i], scale=likelihood_std)
        # Calculate the prior
        prior = norm.logpdf(real_response[i], loc=prior_mu, scale=prior_std)
        # Compute the log of the unnormalized posterior
        log_unnormalized_posterior = likelihood + prior
        # Compute the log of the normalization constant (log of P(S))
        log_normalization_constant = np.log(np.sum(np.exp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std) + norm.logpdf(xs, loc=prior_mu, scale=prior_std)))) + 1e-10
        # Calculate the log-posterior
        log_posterior = log_unnormalized_posterior - log_normalization_constant
        # Update the negative log-likelihood
        nll -= log_posterior
    return nll

# Kay
#  Try minimize() because fmin() doesn't work that well

# Define the wrapped_negative_log_likelihood_prior function (wrapper for prior_negative_log_likelihood)
def wrapped_negative_log_likelihood_prior(params, targ_loc, real_response, prior_mu):
    prior_std, likelihood_std = params
    return prior_negative_log_likelihood(targ_loc, real_response, prior_mu, prior_std, likelihood_std)

# Set initial guesses for prior_std and likelihood_std
init_guess_prior = [10, 10]  # [prior_std_guess, likelihood_std_guess]

In [29]:
# Kay
def negative_log_likelihood_no_prior(targ_loc, real_response, likelihood_std):
    nll = 0
    xs = np.linspace(-90, 90, 361)
    for i in range(targ_loc.shape[0]):
        # Calculate the likelihood
        likelihood = norm.logpdf(real_response[i], loc=targ_loc[i], scale=likelihood_std)
        # Compute the log of the unnormalized posterior
        # Since the prior is uniform, the log of the prior is a constant
        log_unnormalized_posterior = likelihood
        # Compute the log of the normalization constant (log of P(S))
        log_normalization_constant = np.log(np.sum(np.exp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std))))+ 1e-10
        # Calculate the log-posterior
        log_posterior = log_unnormalized_posterior - log_normalization_constant
        # Update the negative log-likelihood
        nll -= log_posterior
    return nll

# Kay
def wrapped_negative_log_likelihood_no_prior(params):
    likelihood_std = params[0]
    return negative_log_likelihood_no_prior(targ_loc, real_response, likelihood_std)

from scipy.optimize import minimize

# Initial guess for likelihood_std
initial_guess_no_prior = np.array([10])

In [35]:
# stability fix
from scipy.special import logsumexp

def negative_log_likelihood(targ_loc, real_response, prior_mu, prior_std, likelihood_std):
    nll = 0
    xs = np.linspace(-90, 90, 361)

    for i in range(targ_loc.shape[0]):
        likelihood = norm.logpdf(real_response[i], loc=targ_loc[i], scale=likelihood_std)
        prior = norm.logpdf(real_response[i], loc=prior_mu, scale=prior_std)
        log_unnormalized_posterior = likelihood + prior

        log_normalization_constant = logsumexp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std) + norm.logpdf(xs, loc=prior_mu, scale=prior_std))

        log_posterior = log_unnormalized_posterior - log_normalization_constant
        nll -= log_posterior

    return nll

def negative_log_likelihood_no_prior(targ_loc, real_response, likelihood_std):
    nll = 0
    xs = np.linspace(-90, 90, 361)

    for i in range(targ_loc.shape[0]):
        likelihood = norm.logpdf(real_response[i], loc=targ_loc[i], scale=likelihood_std)
        log_unnormalized_posterior = likelihood

        log_normalization_constant = logsumexp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std))

        log_posterior = log_unnormalized_posterior - log_normalization_constant
        nll -= log_posterior

    return nll


In [36]:
unique_oriRef2 = data_clean['oriRef2'].unique()
unique_SubjID = data_clean['subjID'].unique()

results = []

for i in unique_SubjID:
    for j in unique_oriRef2:
        subset = data_clean[(data_clean['oriRef2'] == j) & (data_clean['subjID'] == i)]
        targ_loc = np.array(subset['oriFinal2'])
        real_response = np.array(subset['oriRespFinal2'])
        prior_mu = j
        
        # Minimize the negative log-likelihood with prior
        result_prior = minimize(wrapped_negative_log_likelihood_prior, init_guess_prior, args=(targ_loc, real_response, prior_mu), bounds=((1e-5, None), (1e-5, None)))
        estimated_prior_std, estimated_likelihood_std = result_prior.x
        
        nll_prior = result_prior.fun

        # Minimize the negative log-likelihood without prior
        result_no_prior = minimize(wrapped_negative_log_likelihood_no_prior, initial_guess_no_prior)
        estimated_likelihood_std_no_prior = result_no_prior.x[0]
        
        nll_no_prior = result_no_prior.fun

        results.append({
            'subjID': i,
            'oriRef2': j,
            'nll_prior': nll_prior,
            'prior_std': estimated_prior_std,
            'likelihood_std': estimated_likelihood_std,
            'nll_no_prior': nll_no_prior,
            'likelihood_std_no_prior': estimated_likelihood_std_no_prior
        })

results_df = pd.DataFrame(results)
print(results_df)

C:\Users\kay\AppData\Local\Temp\ipykernel_24304\1434391415.py:14: RuntimeWarning: divide by zero encountered in log
  log_normalization_constant = np.log(np.sum(np.exp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std) + norm.logpdf(xs, loc=prior_mu, scale=prior_std)))) + 1e-10
C:\ALL\AppData\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\kay\AppData\Local\Temp\ipykernel_24304\1434391415.py:14: RuntimeWarning: divide by zero encountered in log
  log_normalization_constant = np.log(np.sum(np.exp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std) + norm.logpdf(xs, loc=prior_mu, scale=prior_std)))) + 1e-10
C:\ALL\AppData\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\kay\AppData\Local\Temp\ipykernel_24304\1434391415.py:14: RuntimeWarning: divide by zero encountered in log
  log_normalization_co

C:\Users\kay\AppData\Local\Temp\ipykernel_24304\1434391415.py:14: RuntimeWarning: divide by zero encountered in log
  log_normalization_constant = np.log(np.sum(np.exp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std) + norm.logpdf(xs, loc=prior_mu, scale=prior_std)))) + 1e-10
C:\ALL\AppData\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\kay\AppData\Local\Temp\ipykernel_24304\1434391415.py:14: RuntimeWarning: divide by zero encountered in log
  log_normalization_constant = np.log(np.sum(np.exp(norm.logpdf(xs, loc=targ_loc[i], scale=likelihood_std) + norm.logpdf(xs, loc=prior_mu, scale=prior_std)))) + 1e-10
C:\ALL\AppData\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\kay\AppData\Local\Temp\ipykernel_24304\1434391415.py:14: RuntimeWarning: divide by zero encountered in log
  log_normalization_co

    subjID  oriRef2   nll_prior    prior_std  likelihood_std  nll_no_prior  \
0        1       45  177.447273   697.972763       10.222295    177.446091   
1        1        0  154.399928   786.318709        5.742541           NaN   
2        1       90        -inf    11.451514        8.260103           NaN   
3        1      135        -inf    10.000000       10.000000           NaN   
4        2       45  173.403422    21.420691       10.235522    173.697529   
5        2        0  154.923595   659.333282        5.819454    154.922209   
6        2       90        -inf    10.820620        8.095697     84.970101   
7        2      135        -inf    10.000000       10.000000           NaN   
8        3       45  169.346138   700.221468        8.345089    169.345879   
9        3        0  155.099542  1061.964577        5.843839    155.098680   
10       3       90        -inf    10.294658        7.293084           NaN   
11       3      135        -inf    10.000000       10.000000    

In [37]:
results_df

,subjID,oriRef2,nll_prior,prior_std,likelihood_std,nll_no_prior,likelihood_std_no_prior
0,1,45,177.447273,697.972763,10.222295,177.446091,10.221197
1,1,0,154.399928,786.318709,5.742541,NaN,-7301.606155
2,1,90,-inf,11.451514,8.260103,NaN,-10804.111351
3,1,135,-inf,10.000000,10.000000,NaN,-1024.240000
4,2,45,173.403422,21.420691,10.235522,173.697529,9.303906
5,2,0,154.923595,659.333282,5.819454,154.922209,5.818075
6,2,90,-inf,10.820620,8.095697,84.970101,1.988282
7,2,135,-inf,10.000000,10.000000,NaN,-1024.240000
8,3,45,169.346138,700.221468,8.345089,169.345879,8.344214
9,3,0,155.099542,1061.964577,5.843839,155.098680,5.843799


In [ ]:
# Qingqing
def prior_cal_nll(targ_loc,true_answer, params):
    ntrials = targ_loc.shape[0]
    
    xs = np.arange(-45, 135, 0.01)
    prior_mu = params[0]
    prior_std = params[1]
    likelihood_std = params[2]
    
    # can create prior outside of the loop because doesn't change
    prior = norm.pdf(xs, loc=prior_mu, scale=prior_std)
    
    prior = np.tile(prior[:, np.newaxis], (1, int(ntrials)))

    # representation is also easy to do without a loop
    representation = norm.pdf(xs.reshape(-1,1), loc=targ_loc, scale=likelihood_std)

    post = representation * prior
    post = post/np.sum(post, axis=1, keepdims=True)

    # this is the true answer
    xs_matrix = np.tile(xs, (ntrials,1))
    
    #true_answer = true_answer.values
    
    index = np.argmin(np.abs(xs_matrix - true_answer.reshape(-1,1)), axis=1) #find the closest one
    
    if len(index) != len(true_answer):
        raise ValueError('the num of llh is not the number of trial')

    col_post = post.flatten(order='F')
    col_probs = col_post[index]
    llh = np.sum(np.log(col_probs))
    nLL = -llh 

    return nLL

In [ ]:
# Qingqing
nparams = 3
nsubjs = 15

prior_std_range = np.arange(1, 30, 2)
likelihood_std_range = np.arange(.1,10,.5);


allfits = np.empty((niters, nparams))
nLL_total = np.empty((len(prior_std_range),len(likelihood_std_range)))
prior_std_fit = np.empty((nsubjs))
likelihood_std_fit = np.empty((nsubjs))
the_nLL = np.empty((nsubjs))

for subj in range(nsubjs):
    subject_data =  pd.DataFrame(data_ref2[data_ref2['subjID']==subj+1])
        
    targ_loc = np.array(subject_data['oriFinal'])
    true_answer = np.array(subject_data['oriRespFinal'])
    print(subj)
    for prior_idx in range(len(prior_std_range)):
        for likelihood_idx in range(len(likelihood_std_range)):
            # remember to change the 45 into other prior mu based on the data set u r using
            nll=prior_cal_nll(targ_loc,true_answer,[45, prior_std_range[prior_idx],likelihood_std_range[likelihood_idx]])
            
            nLL_total[prior_idx,likelihood_idx] = nll
    
    the_nLL = np.min(nLL_total) # final fitted nll
    I = np.nanargmin(nLL_total)

    j_idx = np.ceil(I / len(prior_std_range)).astype(int)-1
    i_idx = I - (j_idx) * len(prior_std_range) - 1

    prior_std_fit[subj] = prior_std_range[i_idx] # final fitted prior std
    likelihood_std_fit[subj] = likelihood_std_range[j_idx] # final fitted likelihood std

In [ ]:
# Qingqing
prior_std_fit

In [ ]:
# Qingqing
likelihood_std_fit

In [ ]:
# Qingqing
import matplotlib.pyplot as plt

# create boxplot for prior_std_fit
fig1, ax1 = plt.subplots()
ax1.boxplot(prior_std_fit)
ax1.set_title('Boxplot of Prior Std Fit')
ax1.set_ylabel('Value')

# create boxplot for likelihood_std_fit
fig2, ax2 = plt.subplots()
ax2.boxplot(likelihood_std_fit)
ax2.set_title('Boxplot of Likelihood Std Fit')
ax2.set_ylabel('Value')

plt.show()


#### It's not working now (fmin)
#### I tried to use fmin but the targ_loc i pass in was 40 by 1 after pass in it became 3 by 1 werid.

In [ ]:
# Qingqing
from scipy.optimize import fmin

options = {'ftol': 1e-8, 'disp': False}
nparams = 3
pmin = np.ones(nparams) * 1
pmax = np.ones(nparams) * 50
nsubjs = 15
niters = 1
allfits = np.empty((niters, nparams))
nll = np.empty((niters, 1))
best_params = np.empty((nsubjs, nparams))
min_nll = np.empty((nsubjs, 1))

for subj in range(nsubjs):
    subject_data =  pd.DataFrame(data_ref2[data_ref2['subjID']==subj+1])
    targ_loc = np.array(subject_data['oriFinal'])
    true_answer = np.array(subject_data['oriRespFinal'])
    
    print(targ_loc)
    
    init = np.random.uniform(pmin, pmax, nparams)
    # randomly initialize the starting point
    
    #nll=prior_cal_nll(subject_data, init)    
    #print(nll)
    allfits[iter, :], nll[iter] = fmin(prior_cal_nll, init, args=([targ_loc],[true_answer],), maxiter=500, full_output=True, disp=False)
    # returns the best fit parameters and their values
    
    # do all of that niters times, then pick the best ones
    idx = np.argmin(nll)
    min_nll[subj, :] = nll[idx]
    best_params[subj, :] = allfits[idx, :]
    #print(min_nll[subj,:])
    #print(best_params[subj,:])


### 4. Validation of Prior Model (Recover the params from the simulated data)

In [ ]:
prior_std_range = np.arange(1, 30, 2)
likelihood_std_range = np.arange(.1,10,.5);

allfits = np.empty((niters, nparams))
nLL_retotal = np.empty((len(prior_std_range),len(likelihood_std_range)))
prior_std_refit = np.empty((nsubjs))
likelihood_std_refit = np.empty((nsubjs))
the_renLL = np.empty((nsubjs))

for subj in range(nsubjs):
    subject_data =  pd.DataFrame(data_ref2[data_ref2['subjID']==subj+1])
        
    targ_loc = np.array(subject_data['oriFinal'])
    true_answer = np.array(subject_data['oriRespFinal'])
    print(subj)
    
    prior_mu = 45
    prior_std = prior_std_fit[subj]

    likelihood_std = likelihood_std_fit[subj]

    simu_answers,posterior = prior_simulate_data(targ_loc,prior_mu,prior_std,likelihood_std)
    simu_answers = np.array(simu_answers)
    
    for prior_idx in range(len(prior_std_range)):
        for likelihood_idx in range(len(likelihood_std_range)):
            # remember to change the 45 into other prior mu based on the data set u r using
            nll=prior_cal_nll(targ_loc,simu_answers,[45, prior_std_range[prior_idx],likelihood_std_range[likelihood_idx]])
            
            nLL_retotal[prior_idx,likelihood_idx] = nll
    
    the_renLL[subj] = np.min(nLL_retotal) # final fitted nll
    I = np.nanargmin(nLL_retotal)

    j_idx = np.ceil(I / len(prior_std_range)).astype(int)-1
    i_idx = I - (j_idx) * len(prior_std_range) - 1

    prior_std_refit[subj] = prior_std_range[i_idx]  # refitted prior std
    likelihood_std_refit[subj] = likelihood_std_range[j_idx] # refitted likelihood std

In [ ]:
correlation_data = np.corrcoef(true_answer, simu_answers)[0,1]
correlation_likelihood = np.corrcoef(likelihood_std_refit, likelihood_std_fit)[0,1]
correlation_prior = np.corrcoef(prior_std_fit, prior_std_refit)[0,1]

print([correlation_data, correlation_likelihood, correlation_prior])


In [ ]:
likelihood_std_refit

In [ ]:
likelihood_std_fit

In [ ]:
prior_std_refit

In [ ]:
prior_std_fit

In [ ]:
# Plot the real data, simulated data

# Draw scatter plots and lines
fig, axs = plt.subplots(1, 3, figsize=(15,5))

# True answer and prior fit
axs[0].scatter(true_answer, simu_answers)
m, b = np.polyfit(true_answer, simu_answers, 1)
axs[0].plot(true_answer, m*true_answer + b)
axs[0].set_xlabel('True answer')
axs[0].set_ylabel('Simulated answer')
axs[0].set_title('Correlation = {:.2f}'.format(correlation_data))


#Plot the real data fitted params, and simulated data fitted params

# True and simulated data fitted prior
axs[1].scatter(prior_std_fit, prior_std_refit)
m, b = np.polyfit(prior_std_fit, prior_std_refit, 1)
axs[1].plot(prior_std_fit, m*prior_std_fit + b)
axs[1].set_xlabel('True Answer Fitted Prior std')
axs[1].set_ylabel('Simulated Answer Fitted Prior std')
axs[1].set_title('Correlation = {:.2f}'.format(correlation_prior))

# True and simulated data fitted likelihood
axs[2].scatter(likelihood_std_fit, likelihood_std_refit)
m, b = np.polyfit(likelihood_std_fit, likelihood_std_refit, 1)
axs[2].plot(likelihood_std_fit, m*likelihood_std_fit + b)
axs[2].set_xlabel('True Answer Fitted likelihood std')
axs[2].set_ylabel('Simulated Answer Fitted likelihood std')
axs[2].set_title('Correlation = {:.2f}'.format(correlation_likelihood))

plt.tight_layout()
plt.show()

Model validation 不太好，我感觉可能是simulate有问题，simulate的时候算完posterior之后，我是按那个pdf的频率随机再抽一个数，（另一种选择是直接选posterior最大值对应的点，但那个的simulate结果error都特别小..）或者可能是fit的时候gride search每一步比较大，精确性不高，但现有的计算已经很耗时间了...